In [54]:
import pandas as pd
import joblib   
import numpy as np
from sklearn.model_selection import train_test_split
from SimulateDay import scale_data, predict_action, get_stock_data, add_columns, stock_market_simulation
import xgboost as xgb
import joblib
import altair as alt
import yfinance as yf

def get_stock_movement(symbol):
    data = yf.download(symbol, period="1y")
    if len(data) > 0:
        initial_price = data['Close'].iloc[0]
        latest_price = data['Close'].iloc[-1]
        return ((latest_price - initial_price) / initial_price) * 100
    return 0

import plotly.subplots as sp
import plotly.graph_objects as go
def get_final_portfolio_values(df):
    # Group by 'Stock Name' and get the last row for each group
    final_values = df.groupby('Stock Name').apply(lambda x: x.iloc[-1])
    
    # Extract 'Stock Name' and 'Portfolio Value' columns
    result = final_values[['Stock Name', 'Portfolio Value','Shares Held']].reset_index(drop=True)
    
    return result




In [55]:
sim_results = pd.read_csv('simResults/sim_results.csv')
specific_model_decisions = pd.read_csv('simResults/specific_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])
general_model_decisions = pd.read_csv('simResults/general_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])
LGBM_model_decisions = pd.read_csv('simResults/LGBM_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])
# general_model_decisions.to_csv('simResults/general_model_decisions.csv', index=False)
# specific_model_decisions.to_csv('simResults/specific_model_decisions.csv', index=False)

In [56]:
LGBM_model_decisions.head(11)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
0,A,0,Hold,144.800003,10000.000000,0.0,10000.000000,2024-10-09
1,A,1,Buy,142.740005,9857.259995,1.0,10000.000000,2024-10-10
5755,A,1,Sell,131.781097,10000.000000,0.0,10000.000000,2024-10-24
2,A,2,Hold,143.820007,9857.259995,1.0,10001.080002,2024-10-11
3,A,3,Hold,144.250000,9857.259995,1.0,10001.509995,2024-10-14
4,A,4,Hold,144.580002,9857.259995,1.0,10001.839996,2024-10-15
5,A,5,Buy,139.110001,9718.149994,2.0,9996.369995,2024-10-16
6,A,6,Hold,137.419998,9718.149994,2.0,9992.989990,2024-10-17
7,A,7,Hold,138.634995,9718.149994,2.0,9995.419983,2024-10-18
8,A,8,Sell,136.800003,9854.949997,1.0,9991.750000,2024-10-21


In [57]:
specific_model_decisions.head(11)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
0,A,0,Hold,144.800003,10000.0,0.0,10000.0,2024-10-09
1,A,1,Sell,142.740005,10000.0,0.0,10000.0,2024-10-10
5218,A,1,Sell,133.029999,10000.0,0.0,10000.0,2024-10-23
5720,A,1,Sell,131.781097,10000.0,0.0,10000.0,2024-10-24
2,A,2,Sell,143.820007,10000.0,0.0,10000.0,2024-10-11
3,A,3,Sell,144.250000,10000.0,0.0,10000.0,2024-10-14
4,A,4,Sell,144.580002,10000.0,0.0,10000.0,2024-10-15
5,A,5,Sell,138.830002,10000.0,0.0,10000.0,2024-10-16
6,A,6,Sell,138.245697,10000.0,0.0,10000.0,2024-10-17
7,A,7,Sell,138.774994,10000.0,0.0,10000.0,2024-10-18


In [58]:
LGBM_model_decisions[LGBM_model_decisions['Action'] == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
1,A,1,Buy,142.740005,9857.259995,1.0,10000.000000,2024-10-10
5,A,5,Buy,139.110001,9718.149994,2.0,9996.369995,2024-10-16
16,AAPL,5,Buy,231.779999,9768.220001,1.0,10000.000000,2024-10-16
20,AAPL,9,Buy,234.899994,9533.320007,2.0,10003.119995,2024-10-22
21,AAPL,10,Buy,230.759995,9302.560013,3.0,9994.839996,2024-10-23
...,...,...,...,...,...,...,...,...
5509,ZBRA,9,Buy,372.600006,5522.119995,12.0,9993.320068,2024-10-22
5515,ZTS,4,Buy,192.389999,9807.610001,1.0,10000.000000,2024-10-15
5517,ZTS,6,Buy,191.000000,9616.610001,2.0,9998.610001,2024-10-17
5519,ZTS,8,Buy,189.449997,9427.160004,3.0,9995.509995,2024-10-21


In [59]:
specific_model_decisions[specific_model_decisions['Action']  == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
10,AAPL,0,Buy,229.539993,9770.460007,1.0,10000.000000,2024-10-09
11,AAPL,1,Buy,229.039993,9541.420013,2.0,9999.500000,2024-10-10
47,ACGL,1,Buy,113.900002,9886.099998,1.0,10000.000000,2024-10-10
56,ACN,0,Buy,365.079987,9634.920013,1.0,10000.000000,2024-10-09
5021,ACN,1,Buy,370.630005,9264.290009,2.0,10005.550018,2024-10-23
...,...,...,...,...,...,...,...,...
4960,ZBH,5,Buy,105.209999,9894.790001,1.0,10000.000000,2024-10-16
4961,ZBH,6,Buy,105.029999,9894.970001,1.0,10000.000000,2024-10-17
4962,ZBH,7,Buy,106.535004,9893.464996,1.0,10000.000000,2024-10-18
4963,ZBH,8,Buy,105.239998,9894.760002,1.0,10000.000000,2024-10-21


In [60]:
specific_model_decisions[(specific_model_decisions['Portfolio Value'] > 10000) & (specific_model_decisions['Day'] == specific_model_decisions['Day'].max())].sort_values(by='Portfolio Value', ascending=False)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
608,BKNG,9,Sell,4374.654785,10082.654785,0.0,10082.654785,2024-10-22
4105,SNA,9,Buy,323.440002,9389.109985,2.0,10035.989990,2024-10-22
295,AMP,9,Buy,518.960022,8987.799988,2.0,10025.720032,2024-10-22
2495,ISRG,9,Sell,515.729980,10025.349976,0.0,10025.349976,2024-10-22
2871,LULU,9,Hold,298.225006,9725.880005,1.0,10024.105011,2024-10-22
...,...,...,...,...,...,...,...,...
2631,KEY,9,Sell,17.120001,10000.250000,0.0,10000.250000,2024-10-22
2059,GLW,9,Sell,46.950001,10000.220001,0.0,10000.220001,2024-10-22
738,CAG,9,Buy,29.415001,9941.275000,2.0,10000.105001,2024-10-22
2305,HST,9,Buy,17.495001,9965.054998,2.0,10000.045000,2024-10-22


In [61]:
LGBM_model_decisions[(LGBM_model_decisions['Portfolio Value'] > 10000) & (LGBM_model_decisions['Day'] == LGBM_model_decisions['Day'].max())].sort_values(by='Portfolio Value', ascending=False)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
4113,PKG,10,Hold,228.649994,7807.849991,10.0,10094.349930,2024-10-23
1176,COST,10,Hold,899.169983,4648.799988,6.0,10043.819885,2024-10-23
538,AZO,10,Hold,3196.550049,6844.909912,1.0,10041.459961,2024-10-23
1000,CHTR,10,Hold,332.220001,7379.595032,8.0,10037.355042,2024-10-23
780,BX,10,Buy,166.919998,8701.619980,8.0,10036.979965,2024-10-23
...,...,...,...,...,...,...,...,...
4366,RJF,10,Hold,137.360001,9587.995010,3.0,10000.075012,2024-10-23
3035,KVUE,10,Buy,22.760000,9954.555000,2.0,10000.075001,2024-10-23
1077,CMG,10,Buy,59.020000,9823.000000,3.0,10000.060001,2024-10-23
3453,MNST,10,Hold,53.560001,9839.375000,3.0,10000.055004,2024-10-23


In [62]:
specific_model_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,5989.000000,5989.000000,5989.000000,5989.000000,5989.000000
mean,3.938888,222.547649,9894.947656,0.465585,10000.365525
std,2.927755,523.135956,510.531094,0.772844,9.627753
min,0.000000,7.330000,0.000000,0.000000,9747.400391
25%,1.000000,69.570000,9927.760002,0.000000,10000.000000
50%,4.000000,125.889999,10000.000000,0.000000,10000.000000
75%,7.000000,233.199997,10000.000000,1.000000,10000.000000
max,9.000000,9916.605469,10089.529785,4.000000,10289.005859


In [63]:
LGBM_model_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,6024.000000,6024.000000,6024.000000,6024.000000,6024.000000
mean,4.671979,222.993793,9735.992830,1.188923,9998.674321
std,3.220695,521.595292,708.686855,1.811515,12.572120
min,0.000000,7.330000,0.000000,0.000000,9592.934986
25%,2.000000,69.785000,9763.867504,0.000000,9999.650002
50%,5.000000,126.722500,9995.892498,0.000000,10000.000000
75%,7.250000,233.699997,10000.000000,2.000000,10000.000000
max,10.000000,9924.400391,10041.990234,20.000000,10150.820312


In [64]:
final_portfolio_values = get_final_portfolio_values(sim_results)#.drop(simResults[simResults['Stock Name']=='NFLX'].index))
final_portfolio_values['Profit %'] = (final_portfolio_values['Portfolio Value'] - 10000) / 10000 * 100
alt.Chart(final_portfolio_values).mark_bar().encode(
    x='Stock Name',
    y='Profit %',
    color=alt.condition(
        alt.datum['Profit %'] > 0,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    title='Final Portfolio Value by Stock',
    width=800,
    height=400
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [65]:
final_portfolio_values.describe() 

,Portfolio Value,Shares Held,Profit %
count,18.000000,18.000000,18.000000
mean,10647.323085,34.078928,6.473231
std,990.686666,41.349433,9.906867
min,9902.290070,0.000000,-0.977099
25%,10024.118725,5.250000,0.241187
50%,10161.097538,13.949196,1.610975
75%,10798.232948,51.974158,7.982329
max,13593.265169,122.000000,35.932652


In [66]:
sim_results['Profit %'] = (sim_results['Portfolio Value'] - 10000) / 10000 * 100
sim_results.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value,Profit %
count,4608.000000,4608.000000,4608.000000,4608.000000,4608.000000,4608.000000
mean,127.500000,168.754835,7399.991523,23.691104,10328.447018,3.284470
std,73.908291,165.295578,3568.161207,35.539937,899.489218,8.994892
min,0.000000,11.030000,0.000000,-0.849599,7630.794741,-23.692053
25%,63.750000,42.222500,5792.842522,1.000000,10000.000000,0.000000
50%,127.500000,111.190002,9394.793331,7.990991,10030.187551,0.301876
75%,191.250000,224.577499,9931.619999,30.671116,10199.957554,1.999576
max,255.000000,771.167419,13841.419983,167.000000,15421.850520,54.218505


In [67]:
final_portfolio_values.describe()

,Portfolio Value,Shares Held,Profit %
count,18.000000,18.000000,18.000000
mean,10647.323085,34.078928,6.473231
std,990.686666,41.349433,9.906867
min,9902.290070,0.000000,-0.977099
25%,10024.118725,5.250000,0.241187
50%,10161.097538,13.949196,1.610975
75%,10798.232948,51.974158,7.982329
max,13593.265169,122.000000,35.932652


In [68]:
all_stock_movement = []
total_movement = 0
for stock in sim_results['Stock Name'].unique():
    all_stock_movement.append([stock, get_stock_movement(stock)])
    total_movement += get_stock_movement(stock)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [69]:
total_movement/len(sim_results['Stock Name'].unique())


30.75967264014031

In [70]:
import pandas as pd
sp500 = pd.read_csv('data/sp500_stocks.csv')
sp500 = sp500.sort_values(by=['Symbol','Date'])
sp500 = sp500.drop_duplicates(subset=['Symbol', 'Date'], keep='last')
sp500[sp500['Symbol'] == 'A'].tail(10)

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
3718,2024-10-11,A,143.820007,143.820007,145.059998,143.059998,143.059998,1427400.0
3719,2024-10-14,A,144.250000,144.250000,145.070007,142.820007,144.000000,1765200.0
3720,2024-10-15,A,144.580002,144.580002,146.660004,144.070007,144.350006,1251200.0
3721,2024-10-16,A,139.110001,139.110001,142.565002,138.440002,141.509995,1412079.0
3722,2024-10-17,A,137.419998,137.419998,142.529999,136.479996,141.809998,1592900.0
3723,2024-10-18,A,138.634995,138.634995,139.520004,137.237503,138.529999,821460.0
3724,2024-10-21,A,136.800003,136.800003,138.809998,136.270096,138.350006,302912.0
3725,2024-10-22,A,133.570007,133.570007,134.750000,131.979996,134.690002,709278.0
3726,2024-10-23,A,133.029999,133.029999,134.300003,131.369995,133.270004,1652158.0
1874912,2024-10-24,A,131.781097,131.781097,133.309998,131.509995,133.139999,432792.0
